In [1]:
import os
from pathlib import Path
import time
import json

import pandas as pd
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import (
    StringType,
    IntegerType,
    LongType,
    FloatType,
    DoubleType,
    ShortType,
    DecimalType,
)

os.chdir("..")
print("CWD:", os.getcwd())

BASE_DIR = Path(".").resolve()
DATA_DIR = BASE_DIR / "data"
INDEX_PATH = DATA_DIR / "dataset_index.csv"

CAT_PROFILE_DIR = BASE_DIR / "profiles" / "categorical"
CAT_PROFILE_DIR.mkdir(parents=True, exist_ok=True)

spark = (
    SparkSession.builder
    .appName("NYC_Categorical_Profiling")
    .getOrCreate()
)

index_df = pd.read_csv(INDEX_PATH)
index_records = index_df.to_dict(orient="records")
len(index_records)


CWD: /home/jovyan


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/07 23:55:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/07 23:55:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


40

In [2]:
numeric_types = (
    IntegerType,
    LongType,
    FloatType,
    DoubleType,
    ShortType,
    DecimalType,
)

def get_categorical_columns(sdf, max_distinct_numeric=20):
    string_cols = []
    numeric_low_card = []

    for field in sdf.schema.fields:
        if isinstance(field.dataType, StringType):
            string_cols.append(field.name)

    for field in sdf.schema.fields:
        if isinstance(field.dataType, numeric_types):
            col = field.name
            try:
                distinct_cnt = sdf.select(col).distinct().count()
            except Exception:
                distinct_cnt = None
            if distinct_cnt is not None and distinct_cnt <= max_distinct_numeric:
                numeric_low_card.append(col)

    cols = sorted(set(string_cols + numeric_low_card))
    return cols


In [3]:
profiling_stats = []

for row in index_records:
    if row.get("download_status") != "ok":
        continue

    dataset_id = row["dataset_id"]
    local_path = row["local_path"]
    full_path = str(BASE_DIR / local_path)

    start = time.time()
    status = "ok"
    error = None

    try:
        sdf = (
            spark.read
            .option("header", True)
            .option("inferSchema", True)
            .csv(full_path)
        )
    except Exception as e:
        status = "read_error"
        error = str(e)
        profiling_stats.append(
            {
                "dataset_id": dataset_id,
                "status": status,
                "error": error,
                "seconds": time.time() - start,
                "num_categorical_cols": 0,
            }
        )
        continue

    categorical_cols = get_categorical_columns(sdf)
    categorical_profile = {}

    for col in categorical_cols:
        norm_col = F.lower(F.trim(F.col(col)))
        df_norm = sdf.withColumn(col, norm_col)

        dist = (
            df_norm.groupBy(col)
            .agg(F.count("*").alias("count"))
            .orderBy(F.desc("count"))
        )

        try:
            total_row = dist.agg(F.sum("count").alias("total")).first()
            total = total_row["total"]
        except Exception:
            total = None

        max_values = 100
        dist_rows = dist.limit(max_values).collect()

        top_values = []
        for r in dist_rows:
            value = r[col]
            count = int(r["count"])
            if total:
                percent = float(count) / float(total)
            else:
                percent = None
            top_values.append(
                {
                    "value": value,
                    "count": count,
                    "percent": percent,
                }
            )

        try:
            unique_values = dist.count()
        except Exception:
            unique_values = len(dist_rows)

        categorical_profile[col] = {
            "unique_values": int(unique_values),
            "top_values": top_values,
        }

    feature_groups = {}
    for col in sdf.columns:
        if "_" in col:
            prefix = col.split("_")[0]
        else:
            prefix = col
        feature_groups.setdefault(prefix, []).append(col)

    out_obj = {
        "dataset_id": dataset_id,
        "categorical_profile": categorical_profile,
        "feature_groups": feature_groups,
    }

    out_path = CAT_PROFILE_DIR / f"{dataset_id}_categorical.json"
    with out_path.open("w") as f:
        json.dump(out_obj, f, indent=2)

    elapsed = time.time() - start

    profiling_stats.append(
        {
            "dataset_id": dataset_id,
            "status": status,
            "error": error,
            "seconds": elapsed,
            "num_categorical_cols": len(categorical_cols),
        }
    )


{"ts": "2025-12-07 23:55:45.925", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `SAT Critical Reading Avg`.` Score` cannot be resolved. Did you mean one of the following? [`SAT Critical Reading Avg. Score`, `SAT Writing Avg. Score`, `SAT Math Avg. Score`, `Num of SAT Test Takers`, `SCHOOL NAME`]. SQLSTATE: 42703", "context": {"file": "line 40 in cell [3]", "line": "", "fragment": "col", "errorClass": "UNRESOLVED_COLUMN.WITH_SUGGESTION"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o33.withColumn.\n: org.apache.spark.sql.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `SAT Critical Reading Avg`.` Score` cannot be resolved. Did you mean one of the following? [`SAT Critical Reading Avg. Score`, `SAT Writing Avg. Score`, `SAT Math Avg. Score`, `Num of SAT Test Takers`, `SCHOOL NAME`]. SQLS

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `SAT Critical Reading Avg`.` Score` cannot be resolved. Did you mean one of the following? [`SAT Critical Reading Avg. Score`, `SAT Writing Avg. Score`, `SAT Math Avg. Score`, `Num of SAT Test Takers`, `SCHOOL NAME`]. SQLSTATE: 42703;
'Project [DBN#17, SCHOOL NAME#18, Num of SAT Test Takers#19, 'lower('trim('SAT Critical Reading Avg. Score)) AS SAT Critical Reading Avg. Score#82, SAT Math Avg. Score#21, SAT Writing Avg. Score#22]
+- Relation [DBN#17,SCHOOL NAME#18,Num of SAT Test Takers#19,SAT Critical Reading Avg. Score#20,SAT Math Avg. Score#21,SAT Writing Avg. Score#22] csv


In [ ]:
stats_df = pd.DataFrame(profiling_stats)
stats_df.to_csv(BASE_DIR / "profiles" / "categorical_profiling_times.csv", index=False)
stats_df.head()
